# Imports

In [ ]:
import logging
import os
from pathlib import Path
import geopandas as gpd
import numpy as np
import sys

from multiview_mapping_toolkit.segmentation import (
    write_chips,
)
from multiview_mapping_toolkit.utils.visualization import show_segmentation_labels

sys.path.append("../..")
from constants import (
    LABELS_COLUMN,
    LABELS_FILENAME,
    get_IDs_to_labels,
    get_training_raster_filename,
    get_training_chips_folder,
)

# Define constants 

In [ ]:
TRAINING_SITE = "none"

In [ ]:
CHIP_SIZE = 3648
TRAINING_STRIDE = int(CHIP_SIZE / 2)
INFERENCE_STRIDE = int(CHIP_SIZE / 2)
TRAINING_IMGS_EXT = ".png"
TEST_IMGS_EXT = TRAINING_IMGS_EXT

In [ ]:
logging.basicConfig(level="INFO")

In [ ]:
# Load the data
gdf = gpd.read_file(LABELS_FILENAME)

gdf.query(f"fire=='{TRAINING_SITE}'").plot(LABELS_COLUMN, legend=True, vmin=0, vmax=9)
class_names = np.unique(gpd.read_file(LABELS_FILENAME)[LABELS_COLUMN]).tolist()
class_names

# Create training chips

In [ ]:
IDS_TO_LABELS = get_IDs_to_labels()
TRAINING_RASTER_FILENAME = get_training_raster_filename(training_site=TRAINING_SITE)
TRAINING_CHIPS_FOLDER = get_training_chips_folder(training_site=TRAINING_SITE)

# Interchange the keys and values
label_remap = {v: k for k, v in IDS_TO_LABELS.items()}

write_chips(
    raster_file=TRAINING_RASTER_FILENAME,
    output_folder=TRAINING_CHIPS_FOLDER,
    chip_size=CHIP_SIZE,
    chip_stride=TRAINING_STRIDE,
    label_vector_file=LABELS_FILENAME,
    label_column=LABELS_COLUMN,
    label_remap=label_remap,
    output_suffix=TRAINING_IMGS_EXT,
    ROI_file=LABELS_FILENAME,
)

In [ ]:
show_segmentation_labels(
    label_folder=Path(TRAINING_CHIPS_FOLDER, "anns"),
    image_folder=Path(TRAINING_CHIPS_FOLDER, "imgs"),
    image_suffix=TRAINING_IMGS_EXT,
)